In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_kidney_disease"
cohort = "GSE104948"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_kidney_disease"
in_cohort_dir = "../../input/GEO/Chronic_kidney_disease/GSE104948"

# Output paths
out_data_file = "../../output/preprocess/Chronic_kidney_disease/GSE104948.csv"
out_gene_data_file = "../../output/preprocess/Chronic_kidney_disease/gene_data/GSE104948.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_kidney_disease/clinical_data/GSE104948.csv"
json_path = "../../output/preprocess/Chronic_kidney_disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Glomerular Transcriptome from European Renal cDNA Bank subjects and living donors"
!Series_summary	"summary : Glomerular Transcriptome from European Renal cDNA Bank subjects and living donors. Samples included in this analysis have been previously analyzed using older CDF definitions and are included under previous GEO submissions - GSE47183 (chronic kidney disease samples), and GSE32591 (IgA nephropathy samples).  "
!Series_overall_design	"RNA from the glomerular compartment of was extracted and processed for hybridization on Affymetrix microarrays, annotated using Human Entrez Gene ID custom CDF version 19."
Sample Characteristics Dictionary:
{0: ['tissue: Glomeruli from kidney biopsy'], 1: ['diagnosis: Diabetic Nephropathy', 'diagnosis: Focal Segmental Glomerular Sclerosis/Minimal Change Disease', 'diagnosis: Focal Segmental Glomerular Sclerosis', nan, 'diagnosis: Minimal Change Disease', 'diagnosis: ANCA Associated Vasculitis', 'diagnosis: Tum

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Optional, Callable, Dict, Any, List

# 1. Evaluate gene expression data availability
# From the background information, we can see this is an Affymetrix microarray dataset with gene annotation
# This suggests gene expression data is available
is_gene_available = True

# 2. Analyze sample characteristics for trait, age, and gender availability

# 2.1 Data Availability
# From the sample characteristics dictionary, we can see diagnosis information in index 1
trait_row = 1  # The diagnosis information is available at index 1
age_row = None  # Age information is not available
gender_row = None  # Gender information is not available

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """
    Convert diagnosis values to binary indicating presence of chronic kidney disease (CKD).
    Diabetic Nephropathy, Hypertensive Nephropathy, IgA Nephropathy, FSGS, 
    Membranous Glomerulonephropathy and Lupus Nephritis are all forms of CKD.
    """
    if pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Categorize the different diagnoses
    ckd_diagnoses = [
        'Diabetic Nephropathy', 
        'Hypertensive Nephropathy', 
        'IgA Nephropathy',
        'Focal Segmental Glomerular Sclerosis',
        'Membranous Glomerulonephropathy',
        'Systemic Lupus Erythematosus'  # Lupus can cause lupus nephritis, a form of CKD
    ]
    
    non_ckd_diagnoses = [
        'Minimal Change Disease',  # Typically causes nephrotic syndrome but not chronic kidney failure
        'Thin Membrande Disease',  # Typically benign
        'Tumor Nephrectomy'        # Control samples from tumor-adjacent normal tissue
    ]
    
    if value in ckd_diagnoses:
        return 1  # Has CKD
    elif value in non_ckd_diagnoses:
        return 0  # Does not have CKD
    else:
        return None  # Unknown or ambiguous diagnosis

def convert_age(value):
    """Placeholder function for age conversion."""
    return None

def convert_gender(value):
    """Placeholder function for gender conversion."""
    return None

# 3. Save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Since we have the sample characteristics directly from previous output, 
    # we need to create a DataFrame from this information to pass to geo_select_clinical_features
    sample_chars = {0: ['tissue: Glomeruli from kidney biopsy'], 
                   1: ['diagnosis: Diabetic Nephropathy', 
                       'diagnosis: Minimal Change Disease', 
                       'diagnosis: Thin Membrande Disease', 
                       'diagnosis: Hypertensive Nephropathy', 
                       'diagnosis: Tumor Nephrectomy', 
                       'diagnosis: IgA Nephropathy', 
                       'diagnosis: Focal Segmental Glomerular Sclerosis', 
                       np.nan, 
                       'diagnosis: Membranous Glomerulonephropathy', 
                       'diagnosis: Systemic Lupus Erythematosus']}
    
    # Create a DataFrame from sample characteristics
    # We'll create a transposed structure with columns for each sample and rows for characteristics
    chars_df = pd.DataFrame()
    
    # First, determine how many unique samples we have based on the characteristic values
    # We'll assume each unique value in the diagnosis row represents a different sample
    unique_diagnoses = [val for val in sample_chars[1] if not pd.isna(val)]
    
    # Create a column for each unique diagnosis
    for i, diagnosis in enumerate(unique_diagnoses):
        sample_id = f"GSM{i+1}"
        sample_data = {}
        
        # For each characteristic row, assign the corresponding value
        for row_idx, values_list in sample_chars.items():
            if row_idx == 1:  # Diagnosis row (trait)
                sample_data[row_idx] = diagnosis
            elif row_idx == 0:  # Tissue row
                sample_data[row_idx] = values_list[0]  # Use the first value for all samples
        
        # Add column to dataframe
        chars_df[sample_id] = pd.Series(sample_data)
    
    # Extract clinical features using the provided function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=chars_df,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the output
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save to CSV
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM1': [1.0], 'GSM2': [0.0], 'GSM3': [0.0], 'GSM4': [1.0], 'GSM5': [0.0], 'GSM6': [1.0], 'GSM7': [1.0], 'GSM8': [1.0], 'GSM9': [1.0]}
Clinical data saved to ../../output/preprocess/Chronic_kidney_disease/clinical_data/GSE104948.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


SOFT file: ../../input/GEO/Chronic_kidney_disease/GSE104948/GSE104948_family.soft.gz
Matrix file: ../../input/GEO/Chronic_kidney_disease/GSE104948/GSE104948-GPL22945_series_matrix.txt.gz
Found the matrix table marker in the file.
Gene data shape: (12074, 71)
First 20 gene/probe identifiers:
['10000_at', '10001_at', '10002_at', '10003_at', '100048912_at', '10004_at', '10005_at', '10006_at', '10007_at', '100093698_at', '10009_at', '1000_at', '10010_at', '100126791_at', '100128124_at', '100128640_at', '100129128_at', '100129250_at', '100129271_at', '100129361_at']


### Step 4: Gene Identifier Review

In [5]:
# Looking at the first 20 gene identifiers provided
# These appear to be Affymetrix probe identifiers (e.g., '10000_at', '10001_at')
# The '_at' suffix is typical for Affymetrix arrays
# These are not standard human gene symbols (like BRCA1, TP53, etc.)
# Therefore, mapping to official gene symbols will be required

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Get a more complete view to understand the annotation structure
print("\nComplete sample of a few rows:")
print(gene_annotation.iloc[:3].to_string())

# Check if there are any columns that might contain gene information beyond what we've seen
potential_gene_columns = [col for col in gene_annotation.columns if 
                          any(term in col.upper() for term in ["GENE", "SYMBOL", "NAME", "ID"])]
print(f"\nPotential gene-related columns: {potential_gene_columns}")

# Look for additional columns that might contain gene symbols
# Since we only have 'ID' and 'ENTREZ_GENE_ID', check if we need to use Entrez IDs for mapping
gene_id_col = 'ID'
gene_symbol_col = None

# Check various potential column names for gene symbols
for col_name in ['GENE_SYMBOL', 'SYMBOL', 'GENE', 'GENE_NAME', 'GB_ACC']:
    if col_name in gene_annotation.columns:
        gene_symbol_col = col_name
        break

# If no dedicated symbol column is found, we'll need to use ENTREZ_GENE_ID
if gene_symbol_col is None and 'ENTREZ_GENE_ID' in gene_annotation.columns:
    gene_symbol_col = 'ENTREZ_GENE_ID'
    print("\nNo direct gene symbol column found. Will use Entrez Gene IDs for mapping.")

if gene_id_col in gene_annotation.columns and gene_symbol_col is not None:
    print(f"\nSample mappings from '{gene_id_col}' to '{gene_symbol_col}':")
    sample_mappings = gene_annotation[[gene_id_col, gene_symbol_col]].head(10)
    print(sample_mappings)
    
    # Check for non-null mappings to confirm data quality
    non_null_mappings = gene_annotation[[gene_id_col, gene_symbol_col]].dropna(subset=[gene_symbol_col])
    print(f"\nNumber of probes with gene ID mappings: {len(non_null_mappings)}")
    print(f"Sample of valid mappings:")
    print(non_null_mappings.head(5))
else:
    print("Required mapping columns not found in the annotation data. Will need to explore alternative mapping approaches.")



Gene annotation preview:
Columns in gene annotation: ['ID', 'Symbol', 'SPOT_ID', 'ENTREZ_GENE_ID']
{'ID': ['1000_at', '10000_at', '100009676_at', '10001_at', '10004_at'], 'Symbol': ['CDH2', 'AKT3', 'ZBTB11-AS1', 'MED6', 'NAALADL1'], 'SPOT_ID': ['cadherin 2', 'AKT serine/threonine kinase 3', 'ZBTB11 antisense RNA 1', 'mediator complex subunit 6', 'N-acetylated alpha-linked acidic dipeptidase-like 1'], 'ENTREZ_GENE_ID': ['1000', '10000', '100009676', '10001', '10004']}

Complete sample of a few rows:
             ID      Symbol                        SPOT_ID ENTREZ_GENE_ID
0       1000_at        CDH2                     cadherin 2           1000
1      10000_at        AKT3  AKT serine/threonine kinase 3          10000
2  100009676_at  ZBTB11-AS1         ZBTB11 antisense RNA 1      100009676

Potential gene-related columns: ['ID', 'Symbol', 'SPOT_ID', 'ENTREZ_GENE_ID']

No direct gene symbol column found. Will use Entrez Gene IDs for mapping.

Sample mappings from 'ID' to 'ENTREZ_GENE_ID

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in gene annotation store the gene identifiers and symbols
gene_id_col = 'ID'  # The probe identifiers (e.g., 10000_at)
gene_symbol_col = 'Symbol'  # The column containing gene symbols (e.g., AKT3)

# 2. Extract the mapping between probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, gene_id_col, gene_symbol_col)
print(f"Gene mapping shape: {gene_mapping.shape}")
print("Sample of gene mapping:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert from probe-level to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene data shape after mapping: {gene_data.shape}")
print("First few genes after mapping:")
print(gene_data.index[:10].tolist())

# Normalize gene symbols to handle synonyms and aggregate values for same genes
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")

# Check how many unique genes we have in the processed data
print(f"Number of unique genes in the processed data: {len(gene_data.index)}")

# Save the gene expression data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")


Gene mapping shape: (2398539, 2)
Sample of gene mapping:
             ID        Gene
0       1000_at        CDH2
1      10000_at        AKT3
2  100009676_at  ZBTB11-AS1
3      10001_at        MED6
4      10004_at    NAALADL1


Gene data shape after mapping: (11996, 71)
First few genes after mapping:
['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAGAB', 'AAK1', 'AAMDC']
Gene data shape after normalization: (11965, 71)
Number of unique genes in the processed data: 11965


Gene data saved to ../../output/preprocess/Chronic_kidney_disease/gene_data/GSE104948.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    print(f"Loaded gene data shape: {gene_data.shape}")
    
    # Gene normalization was already applied in the previous step
    print(f"Gene data already normalized with shape: {gene_data.shape}")
    is_gene_available = True
except Exception as e:
    print(f"Error loading gene data: {e}")
    is_gene_available = False

# 2. Load the original data sources to properly extract sample IDs and clinical data
try:
    # Get file paths
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Extract properly formatted clinical data directly from source
    background_info, clinical_df = get_background_and_clinical_data(matrix_file)
    
    # Check the first few rows to understand the data structure
    print("Clinical data from source:")
    print(clinical_df.head(3))
    
    # Get the sample accession IDs that will match gene data column names
    sample_ids = clinical_df['!Sample_geo_accession'].tolist()
    print(f"Found {len(sample_ids)} sample IDs in the matrix file")
    print(f"Sample IDs (first 5): {sample_ids[:5]}")
    
    # Create proper clinical data with trait information
    trait_data = geo_select_clinical_features(
        clinical_df=clinical_df,
        trait=trait,
        trait_row=1,  # Based on previous analysis
        convert_trait=convert_trait,
        age_row=None,
        convert_age=None,
        gender_row=None,
        convert_gender=None
    )
    
    print(f"Extracted clinical data shape: {trait_data.shape}")
    print("Preview of trait data:")
    print(trait_data.head())
    
    # Save the properly extracted clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    trait_data.to_csv(out_clinical_data_file)
    print(f"Updated clinical data saved to {out_clinical_data_file}")
    
    is_trait_available = True
except Exception as e:
    print(f"Error extracting clinical data: {e}")
    is_trait_available = False

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Check that gene_data columns match sample_ids from clinical data
        common_samples = set(gene_data.columns).intersection(trait_data.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Filter gene data to include only common samples
            gene_data_filtered = gene_data[list(common_samples)]
            # Filter trait data to include only common samples
            trait_data_filtered = trait_data[list(common_samples)]
            
            # Perform the linking
            linked_data = geo_link_clinical_genetic_data(trait_data_filtered, gene_data_filtered)
            print(f"Linked data shape: {linked_data.shape}")
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            # Check for bias in trait and demographic features
            is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
            
            # Validate the data quality and save cohort info
            note = "Dataset contains gene expression data from kidney glomeruli of patients with various kidney conditions including diabetic nephropathy, IgA nephropathy, and other kidney diseases."
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=is_biased,
                df=linked_data,
                note=note
            )
            
            # Save the linked data if it's usable
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                linked_data.to_csv(out_data_file)
                print(f"Linked data saved to {out_data_file}")
            else:
                print("Data not usable for the trait study - not saving final linked data.")
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=pd.DataFrame(),  # Empty dataframe for metadata
        note="Missing essential data components for linking (trait data or gene expression data)."
    )

Loaded gene data shape: (11965, 71)
Gene data already normalized with shape: (11965, 71)
Clinical data from source:
         !Sample_geo_accession                            GSM2810770  \
0  !Sample_characteristics_ch1  tissue: Glomeruli from kidney biopsy   
1  !Sample_characteristics_ch1       diagnosis: Diabetic Nephropathy   

                             GSM2810771                            GSM2810772  \
0  tissue: Glomeruli from kidney biopsy  tissue: Glomeruli from kidney biopsy   
1       diagnosis: Diabetic Nephropathy       diagnosis: Diabetic Nephropathy   

                             GSM2810773                            GSM2810774  \
0  tissue: Glomeruli from kidney biopsy  tissue: Glomeruli from kidney biopsy   
1       diagnosis: Diabetic Nephropathy       diagnosis: Diabetic Nephropathy   

                             GSM2810775                            GSM2810776  \
0  tissue: Glomeruli from kidney biopsy  tissue: Glomeruli from kidney biopsy   
1       diagnosis

Linked data shape after handling missing values: (25, 11966)
For the feature 'Chronic_kidney_disease', the least common label is '0.0' with 8 occurrences. This represents 32.00% of the dataset.
The distribution of the feature 'Chronic_kidney_disease' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Chronic_kidney_disease/cohort_info.json


Linked data saved to ../../output/preprocess/Chronic_kidney_disease/GSE104948.csv
